In [0]:
%pylab inline

In [24]:
import dataiku
import pandas as pd, numpy as np
from dataiku import pandasutils as pdu
import requests
from bs4 import BeautifulSoup
import csv
from datetime import datetime as dt
from datetime import datetime, timedelta

place_code = {"東京":{"prec_no":"44", "block_no":"47662", "type":"s1"}, "大分":{"prec_no":"83", "block_no":"0808", "type":"a1"}, "弘前":{"prec_no":"31", "block_no":"0166", "type":"a1"}}
base_url = "http://www.data.jma.go.jp/obd/stats/etrn/view/daily_%s.php?prec_no=%s&block_no=%s&year=%s&month=%s&day=&view=p1"

def str2float(str):
    try:
        return float(str)
    except:
        return 0.0

l = []

today = dt.today() - timedelta(days=0) # - 1 #前日の気温を取得するため
day = int(today.day) 
month = int(today.month)
year = int(today.year)

In [25]:
year, month, day

(2021, 3, 10)

In [26]:
for place in place_code.keys():
    print(place)
    prec_no = place_code[place]["prec_no"]
    block_no = place_code[place]["block_no"]
    data_type = place_code[place]["type"]

    year = year
    url = base_url%(data_type, prec_no, block_no, year, month)
    print(url)
    r = requests.get(url)
    r.encoding = r.apparent_encoding

    soup = BeautifulSoup(r.text)
    rows = soup.findAll('tr',class_='mtx')

    # 大分と弘前のページ
    if data_type == "a1":
        row = rows[3:][day-1]
        data = row.findAll('td')
        date = str(year) + "/" + str(month) + "/" + str(day)
        precip_total  = data[1].string #降水量(mm)合計
        temp_ave = data[4].string #平均気温
        temp_high = data[5].string #最高気温
        temp_low = data[6].string #最低気温
        daylight = data[13].string #日照時間
        d = {"date":date, "precip_total":precip_total, "temp_ave":temp_ave, "temp_high":temp_high, "temp_low":temp_low, "daylight":daylight, "place":place}
        l.append(d)
        
    # 東京のページ（大分、弘前とはテーブルのフォーマットが微妙に違う）
    else:
        row = rows[4:][day-1]
        data = row.findAll('td')
        date = str(year) + "/" + str(month) + "/" + str(day)
        precip_total  = data[3].string #降水量(mm)合計
        temp_ave = data[6].string #平均気温
        temp_high = data[7].string #最高気温
        temp_low = data[8].string #最低気温
        daylight = data[16].string #日照時間
        d = {"date":date, "precip_total":precip_total, "temp_ave":temp_ave, "temp_high":temp_high, "temp_low":temp_low, "daylight":daylight, "place":place}
        l.append(d)

df = pd.DataFrame(l)

todays_weather = dataiku.Dataset("today_weather")
todays_weather.write_with_schema(df)

弘前
http://www.data.jma.go.jp/obd/stats/etrn/view/daily_a1.php?prec_no=31&block_no=0166&year=2021&month=3&day=&view=p1
東京
http://www.data.jma.go.jp/obd/stats/etrn/view/daily_s1.php?prec_no=44&block_no=47662&year=2021&month=3&day=&view=p1
大分
http://www.data.jma.go.jp/obd/stats/etrn/view/daily_a1.php?prec_no=83&block_no=0808&year=2021&month=3&day=&view=p1
3 rows successfully written (duNBjPGkfg)


In [15]:
base_url%(data_type, prec_no, block_no, year, month)

'http://www.data.jma.go.jp/obd/stats/etrn/view/daily_a1.php?prec_no=31&block_no=0166&year=2021&month=2&day=&view=p1'

In [21]:
rows[3:][0]

<tr class="mtx" style="text-align:right;"><td style="white-space:nowrap"><div class="a_print"><a href="hourly_a1.php?prec_no=31&amp;block_no=0166&amp;year=2021&amp;month=02&amp;day=1&amp;view=p1">1</a></div></td><td class="data_0_0">0.5</td><td class="data_0_0">0.5</td><td class="data_0_0">0.5</td><td class="data_0_0">0.4</td><td class="data_0_0">7.7</td><td class="data_0_0">-10.2</td><td class="data_0_0">2.2</td><td class="data_0_0">5.7</td><td class="data_0_0" style="text-align:center">\u5357\u5357\u897f</td><td class="data_0_0">12.2</td><td class="data_0_0" style="text-align:center">\u5357</td><td class="data_0_0" style="text-align:center">\u5357\u5357\u897f</td><td class="data_0_0">6.1</td><td class="data_0_0">0</td><td class="data_0_0">46</td></tr>